# 02_baseline_rule_based — Binario A/D

**Objetivo:** baseline **rule-based** usando el fork del proyecto colombiano (solo **Ansiedad/Depresión**) para obtener una primera línea de referencia.  
**Justificación:** las reglas permiten:
- establecer un punto de partida interpretable (trazabilidad por JSON/patrones),
- detectar fallos sistemáticos del dataset (typos, negación, expresiones locales),
- guiar el diseño del *cleaning* y la selección de fenotipos relevantes para A/D.

> Nota: mantenemos **preprocesamiento ligero** para no romper *ConText* ni *TargetMatcher*.


In [6]:
# ===============================================================
# Setup: Paths, Imports, y Utilidades Compartidas
# ===============================================================

from pathlib import Path
import pandas as pd
import re, unicodedata, os

# Intentar importar utilidades compartidas
try:
    import sys
    sys.path.insert(0, str(Path.cwd().parent))
    from utils_shared import setup_paths, guess_text_col, guess_label_col, normalize_label
    print("[INFO] Utilizando utils_shared.py")
    
    # Setup de paths centralizado
    paths = setup_paths()
    BASE_PATH = paths['BASE_PATH']
    DATA_PATH = paths['DATA_PATH']
    FORK_PATH = paths['FORK_PATH']
    SPLITS_PATH = paths['SPLITS_PATH']
    
    # Usar funciones centralizadas
    _guess_text_col = guess_text_col
    _guess_label_col = guess_label_col
    _norm_label_bin = normalize_label
    
except ImportError:
    print("[WARNING] utils_shared.py no encontrado, usando funciones locales")
    
    # Setup manual de paths
    BASE_PATH = Path.cwd()
    if BASE_PATH.name == "notebooks":
        BASE_PATH = BASE_PATH.parent
    
    DATA_PATH = BASE_PATH / "data"
    FORK_PATH = BASE_PATH / "Spanish_Psych_Phenotyping_PY"
    SPLITS_PATH = DATA_PATH / "splits"
    
    DATA_PATH.mkdir(exist_ok=True)
    
    # Funciones helper locales
    def _guess_text_col(df):
        for c in ["texto", "text", "comment", "comentario"]:
            if c in df.columns:
                return c
        return df.columns[0]
    
    def _guess_label_col(df):
        for c in ["etiqueta", "label", "category"]:
            if c in df.columns:
                return c
        return df.columns[1] if len(df.columns) > 1 else df.columns[-1]
    
    def _norm_label_bin(s):
        if pd.isna(s): 
            return ""
        s = str(s).strip().lower()
        s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
        return {'depresivo': 'depresion'}.get(s, s)

# Validar existencia de directorios críticos
if not FORK_PATH.exists():
    raise FileNotFoundError(
        f"[ERROR] Fork no encontrado en {FORK_PATH}\n"
        f"        Este baseline requiere Spanish_Psych_Phenotyping_PY/"
    )

if not SPLITS_PATH.exists():
    raise FileNotFoundError(
        f"[ERROR] Splits no encontrados en {SPLITS_PATH}\n"
        f"        Debes ejecutar primero: 02_create_splits.ipynb"
    )

print(f"[INFO] Paths configurados:")
print(f"  BASE_PATH:   {BASE_PATH}")
print(f"  DATA_PATH:   {DATA_PATH}")
print(f"  FORK_PATH:   {FORK_PATH}")
print(f"  SPLITS_PATH: {SPLITS_PATH}")

# Columnas esperadas en dataset_base.csv
TEXT_COL = "texto"
LABEL_COL = "etiqueta"

[INFO] Utilizando utils_shared.py
[INFO] Paths configurados:
  BASE_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay
  DATA_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data
  FORK_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay/Spanish_Psych_Phenotyping_PY
  SPLITS_PATH: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits


## 1) Carga y preprocesamiento **ligero** (conserva tildes y casing, colapsa alargamientos)

In [7]:
# ===============================================================
# CARGA DE DATOS - PATIENT-LEVEL SPLIT (SIN LEAKAGE)
# ===============================================================
# IMPORTANTE: Este baseline usa splits generados por 02_create_splits.ipynb
#
# Estrategia de split:
#   - Por PACIENTES (no por casos/consultas)
#   - 72 pacientes train / 18 pacientes val
#   - 0% overlap (sin data leakage)
#
# ¿Por qué patient-level?
#   Dataset tiene estructura longitudinal: 90 pacientes × 35 consultas promedio
#   Split por casos → 100% pacientes en train Y val (leakage total)
#   Split por pacientes → 0% overlap (generaliza a pacientes nuevos)
#
# Ver detalles en: ESTRATEGIA_SPLIT_PACIENTES.md

import pandas as pd, re, unicodedata

# Cargar splits unificados desde 02_create_splits.ipynb
print("="*60)
print("CARGA DE SPLITS (PATIENT-LEVEL)")
print("="*60)

dataset_base = pd.read_csv(SPLITS_PATH / 'dataset_base.csv')
train_indices = pd.read_csv(SPLITS_PATH / 'train_indices.csv')['row_id'].values
val_indices = pd.read_csv(SPLITS_PATH / 'val_indices.csv')['row_id'].values

print(f"✓ Dataset base: {len(dataset_base)} casos")
print(f"✓ Train indices: {len(train_indices)} casos")
print(f"✓ Val indices: {len(val_indices)} casos")

text_col = _guess_text_col(dataset_base)
label_col = _guess_label_col(dataset_base)

# ===============================================================
# PREPROCESAMIENTO LIGERO (conserva estructura para rule-based)
# ===============================================================
# Estrategia: Mínima normalización (preserva tildes, mayúsculas, puntuación)
# - Los patrones JSON son case-sensitive y usan tildes
# - Solo colapsa alargamientos para mantener matching
# - Comparación: TF-IDF normaliza todo, BETO tokeniza, Rule-based conserva estructura

_RE_MULTI = re.compile(r'(.)\1{2,}')  # Detecta 3+ letras repetidas

def clean_text_rb(s: str) -> str:
    """
    Limpieza LIGERA para rule-based (conserva estructura original).
    
    Aplica solo:
    - Normalización NFC (forma canónica de tildes)
    - Colapso de alargamientos (holaaa → holaa)
    - Normalización de espacios
    """
    if pd.isna(s):
        return ""
    
    s = str(s).strip()
    s = unicodedata.normalize("NFC", s)  # Normaliza tildes (é = é, no e + ´)
    s = _RE_MULTI.sub(r'\1\1', s)        # holaaa → holaa (mantiene énfasis)
    s = re.sub(r"\s+", " ", s).strip()   # Colapsa espacios múltiples
    
    return s

dataset_base['texto_rb'] = dataset_base[text_col].map(clean_text_rb)

# Filtrar por índices (patient-level split)
df_train = dataset_base[dataset_base['row_id'].isin(train_indices)].copy()
df_val = dataset_base[dataset_base['row_id'].isin(val_indices)].copy()

print(f"\n[INFO] Split aplicado (patient-level):")
print(f"  Train: {len(df_train)} casos")
print(f"  Val:   {len(df_val)} casos")
print(f"\n[INFO] Distribución train: {dict(df_train[label_col].value_counts())}")
print(f"[INFO] Distribución val: {dict(df_val[label_col].value_counts())}")
print("\n  RECORDATORIO: Estos splits eliminan leakage (pacientes disjuntos)")
print("   Métricas serán más conservadoras pero generalizan mejor.")

CARGA DE SPLITS (PATIENT-LEVEL)
✓ Dataset base: 3155 casos
✓ Train indices: 2509 casos
✓ Val indices: 646 casos

[INFO] Split aplicado (patient-level):
  Train: 2509 casos
  Val:   646 casos

[INFO] Distribución train: {'depresion': 1745, 'ansiedad': 764}
[INFO] Distribución val: {'depresion': 485, 'ansiedad': 161}

  RECORDATORIO: Estos splits eliminan leakage (pacientes disjuntos)
   Métricas serán más conservadoras pero generalizan mejor.

[INFO] Split aplicado (patient-level):
  Train: 2509 casos
  Val:   646 casos

[INFO] Distribución train: {'depresion': 1745, 'ansiedad': 764}
[INFO] Distribución val: {'depresion': 485, 'ansiedad': 161}

  RECORDATORIO: Estos splits eliminan leakage (pacientes disjuntos)
   Métricas serán más conservadoras pero generalizan mejor.


## 2) Ejecutar fork (perfil `col`) con solo Ansiedad/Depresión

In [8]:
import sys, subprocess, yaml
from pathlib import Path

cfg_dir = FORK_PATH/'configs'
cfg_dir.mkdir(parents=True, exist_ok=True)
col_cfg = cfg_dir/'col_config.yml'
fenos_yml = cfg_dir/'fenotipos.yml'

# Forzar solo Ansiedad/Depresion en el fork
cfg = {}
if col_cfg.exists():
    cfg = yaml.safe_load(col_cfg.read_text(encoding='utf-8')) or {}
cfg['text_column'] = 'texto_rb'
col_cfg.write_text(yaml.safe_dump(cfg, allow_unicode=True), encoding='utf-8')

fen = {}
if fenos_yml.exists():
    fen = yaml.safe_load(fenos_yml.read_text(encoding='utf-8')) or {}
fen['active_concepts'] = ['Ansiedad','Depresion']
fenos_yml.write_text(yaml.safe_dump(fen, allow_unicode=True), encoding='utf-8')

cli_py = FORK_PATH/'cli.py'
main_py = FORK_PATH/'main.py'
runner = cli_py if cli_py.exists() else main_py
assert runner.exists(), "No se encontró cli.py ni main.py en el fork."

# Crear temp input solo con val set (para evaluar)
tmp_in = DATA_PATH/'ips_clean_tmp.csv'
df_val[['texto_rb', label_col]].rename(columns={'texto_rb':'texto_rb'}).to_csv(tmp_in, index=False, encoding='utf-8')

# Salidas estandarizadas (comparables)
rule_pred_csv   = DATA_PATH/'rule_based_predictions.csv'
rule_report_csv = DATA_PATH/'rule_based_classification_report.csv'
rule_eval_csv   = DATA_PATH/'rule_based_eval.csv'
rule_cm_csv     = DATA_PATH/'rule_based_confusion_matrix.csv'

cmd = [sys.executable, str(runner), '--profile','col', '--config', str(col_cfg),
       '--input', str(tmp_in), '--output', str(rule_pred_csv)]
print("CMD:", " ".join(map(str,cmd)))
ret = subprocess.run(cmd, check=False, capture_output=True, text=True)
print(ret.stdout)
if ret.returncode != 0:
    print(ret.stderr)
    raise RuntimeError(f"CLI terminó con código {ret.returncode}")

CMD: /opt/anaconda3/bin/python /Users/manuelnunez/Projects/psych-phenotyping-paraguay/Spanish_Psych_Phenotyping_PY/cli.py --profile col --config /Users/manuelnunez/Projects/psych-phenotyping-paraguay/Spanish_Psych_Phenotyping_PY/configs/col_config.yml --input /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/ips_clean_tmp.csv --output /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/rule_based_predictions.csv
Components in NLP pipeline:
	- tok2vec
	- morphologizer
	- attribute_ruler
	- lemmatizer
	- medspacy_pyrush
	- medspacy_target_matcher
	- medspacy_context
Concepts included (by folder): Ansiedad, Depresion
Rule categories loaded: Abulia, Agitacinpsicomotora, AngustiaMiedoTemor, Anhedonia, Animodeprimido, Ansiedad, Apata, Apetitoaumentode, Apetitodisminucinde, Autolesin, Bajaconcentracin, Bajaenerga, Compulsiones, Culpa, Desesperanza, DespersonalizacinDesrealizacin, Disforia, Fatiga, Hipotimia, Ideacinpersecutoria, Ideacinsuicida, Ideasdemuerte, Intentosuicida,

## 3) Evaluación **binaria** y exportables

In [9]:
import pandas as pd, unicodedata as _ud
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix

preds = pd.read_csv(rule_pred_csv)
if 'pred_label' not in preds.columns:
    raise ValueError("El output del fork no contiene 'pred_label'.")

def _norm_txt(s):
    if pd.isna(s): return ""
    s = str(s).strip().lower()
    s = _ud.normalize("NFKD", s).encode("ascii","ignore").decode("ascii")
    return s

y_pred = preds['pred_label'].map(_norm_txt)
y_true = df_val[label_col].map(_norm_txt)

# En caso de que el fork devuelva algo fuera de A/D, lo mapeamos a la clase mayoritaria para evaluar binario
allowed = {'ansiedad','depresion'}
majority = y_true.value_counts().idxmax()
y_pred = y_pred.where(y_pred.isin(allowed), majority)

classes = ['depresion','ansiedad']

pd.DataFrame(classification_report(y_true, y_pred, labels=classes, output_dict=True, zero_division=0)).transpose()  .to_csv(rule_report_csv, index=True, encoding='utf-8')

pd.DataFrame([{
    'macro_f1': f1_score(y_true, y_pred, average='macro', zero_division=0),
    'macro_precision': precision_score(y_true, y_pred, average='macro', zero_division=0),
    'macro_recall': recall_score(y_true, y_pred, average='macro', zero_division=0),
    'n': int(len(y_true))
}]).to_csv(rule_eval_csv, index=False, encoding='utf-8')

cm = confusion_matrix(y_true, y_pred, labels=classes)
pd.DataFrame(cm, index=[f'true_{c}' for c in classes], columns=[f'pred_{c}' for c in classes]).to_csv(rule_cm_csv)

print("[INFO] Exportados:")
print("  - Predicciones:", rule_pred_csv)
print("  - Reporte:", rule_report_csv)
print("  - Métricas:", rule_eval_csv)
print("  - Matriz:", rule_cm_csv)

[INFO] Exportados:
  - Predicciones: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/rule_based_predictions.csv
  - Reporte: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/rule_based_classification_report.csv
  - Métricas: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/rule_based_eval.csv
  - Matriz: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/rule_based_confusion_matrix.csv


## 4) Análisis de Errores (FP/FN)

In [10]:
# Exportar errores para análisis cualitativo
fp_depresion = df_val[(y_true == 'ansiedad') & (y_pred == 'depresion')].copy()
fp_depresion['error_type'] = 'FP_depresion'

fn_depresion = df_val[(y_true == 'depresion') & (y_pred == 'ansiedad')].copy()
fn_depresion['error_type'] = 'FN_depresion'

fp_ansiedad = df_val[(y_true == 'depresion') & (y_pred == 'ansiedad')].copy()
fp_ansiedad['error_type'] = 'FP_ansiedad'

fn_ansiedad = df_val[(y_true == 'ansiedad') & (y_pred == 'depresion')].copy()
fn_ansiedad['error_type'] = 'FN_ansiedad'

rule_fp_dep_csv = DATA_PATH / 'rule_based_fp_depresion.csv'
rule_fn_dep_csv = DATA_PATH / 'rule_based_fn_depresion.csv'
rule_fp_ans_csv = DATA_PATH / 'rule_based_fp_ansiedad.csv'
rule_fn_ans_csv = DATA_PATH / 'rule_based_fn_ansiedad.csv'

fp_depresion[['texto_rb', label_col, 'error_type']].to_csv(rule_fp_dep_csv, index=False, encoding='utf-8')
fn_depresion[['texto_rb', label_col, 'error_type']].to_csv(rule_fn_dep_csv, index=False, encoding='utf-8')
fp_ansiedad[['texto_rb', label_col, 'error_type']].to_csv(rule_fp_ans_csv, index=False, encoding='utf-8')
fn_ansiedad[['texto_rb', label_col, 'error_type']].to_csv(rule_fn_ans_csv, index=False, encoding='utf-8')

print("[INFO] Análisis de errores exportado:")
print(f"  FP Depresión: {len(fp_depresion)} casos → {rule_fp_dep_csv.name}")
print(f"  FN Depresión: {len(fn_depresion)} casos → {rule_fn_dep_csv.name}")
print(f"  FP Ansiedad:  {len(fp_ansiedad)} casos → {rule_fp_ans_csv.name}")
print(f"  FN Ansiedad:  {len(fn_ansiedad)} casos → {rule_fn_ans_csv.name}")

[INFO] Análisis de errores exportado:
  FP Depresión: 99 casos → rule_based_fp_depresion.csv
  FN Depresión: 0 casos → rule_based_fn_depresion.csv
  FP Ansiedad:  0 casos → rule_based_fp_ansiedad.csv
  FN Ansiedad:  99 casos → rule_based_fn_ansiedad.csv


/var/folders/25/fy01l91x3gj63g090ghxj7000000gn/T/ipykernel_1040/2687782774.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fp_depresion = df_val[(y_true == 'ansiedad') & (y_pred == 'depresion')].copy()
/var/folders/25/fy01l91x3gj63g090ghxj7000000gn/T/ipykernel_1040/2687782774.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fn_depresion = df_val[(y_true == 'depresion') & (y_pred == 'ansiedad')].copy()
/var/folders/25/fy01l91x3gj63g090ghxj7000000gn/T/ipykernel_1040/2687782774.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fp_ansiedad = df_val[(y_true == 'depresion') & (y_pred == 'ansiedad')].copy()
/var/folders/25/fy01l91x3gj63g090ghxj7000000gn/T/ipykernel_1040/2687782774.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fn_ansiedad = df_val[(y_true == 'ansiedad') & (y_pred == 'depresion')].copy()
